In [42]:
# -*- coding: utf-8 -*-
# Imports

"Imports"
import pandas as pd# for dataframe related operations
import numpy as np # for operations like ceil, log, and random.normal
from statsmodels.graphics.tsaplots import plot_acf, acf
import matplotlib.pyplot as plt # for plotting
from sklearn.preprocessing import OneHotEncoder

In [43]:
# The following ensures that all plots are displayed inline
%matplotlib inline
import os
print(os.getcwd())
os.listdir(os.getcwd())

/home/james/Desktop


['SGX-Full-OrderBook-Tick-Data-Trading-Strategy-master.zip',
 'Producers15.pdf',
 'ESH5.Futures.FullDepth.20150128.csv',
 '.~lock.Legacy Tactical Meeting 2.docx#',
 'Machine Learning Group Work Submission 3.docx',
 'Producers14.pdf',
 'Datasets',
 'stocklook-master',
 'Horizon Returns.csv',
 'RiskSummary.csv',
 'Nifty 50 Futures Historical Data.csv',
 'CapstoneProject_HFT.ipynb',
 'Machine Learning Group Work Submission 1.zip',
 '448Project-master.zip',
 'Legacy',
 'appendix 2-GWP Sub1.ipynb',
 'BSEStockData-master',
 'Machine Learning Group Work Submission 3.pdf',
 'ML_GroupWork_Submission3.py',
 '.ipynb_checkpoints',
 'BSE Sensex Futures Historical Data 2020.csv',
 '.~lock.Daily Production Report (12).xlsx#',
 'strategyreturns.csv',
 'BidAskSpread-master',
 'Untitled.ipynb',
 'MachineLearning_GroupWork_Submission3.ipynb',
 'AbsolutePerformance.png',
 'WQU_CF_Module1_Notebook',
 'WQU_MLF_Module 4_Collaborative Review Task.zip',
 'GroupWorkProject sub#3.ipynb',
 'ProximityHostingData.x

In [44]:
# Reading data from a csv in the working directory to a Pandas DataFrame
tick_data = pd.read_csv("BSE Sensex Futures Historical Data 2020.csv")
tick_data.tail()

Date     Price      Open      High       Low    Vol. Change %
1231  01/07/15  27019.00  27077.50  27150.50  26879.00  65.37K   -0.22%
1232  01/06/15  27079.25  27700.00  27700.00  27035.75  55.37K   -3.25%
1233  01/05/15  27990.25  28111.25  28214.00  27937.00  44.34K   -0.34%
1234  01/02/15  28086.00  27780.50  28129.25  27768.50  46.47K    1.42%
1235  01/01/15  27692.50  27610.00  27721.50  27569.75  13.29K   -0.01%

In [45]:
"Pre-processing"
tick_data = asset_prices.rename(columns={'Adj Close':'AdjClose'})

NameError: name 'asset_prices' is not defined

In [39]:
import plotly.graph_objects as go # for plotting candlestick charts
import plotly.express as go
# P.S.1: Must run this to view the plots: conda install -c plotly plotly-orca psutil requests
# P.S.2: Default renderer = 'png': plotly.io.renderer.default = "png"

NameError: name 'asset_prices' is not defined

In [ ]:
asset_prices['Return'] = np.log(asset_prices['AdjClose']).diff()
asset_prices = asset_prices.dropna()

In [20]:
# CREATING VOLUME BARS

# Cumulative volume column is used to bin data in units of 21731
tick_data['CumVolume'] = tick_data['Volume'].cumsum() # calculating cumulative volume
binned_tick_data = bin_data(tick_data,'CumVolume',units = 21731) # binning data by calling bin_data function defined earlier
volume_bars = extract_data(binned_tick_data) # extracting open, high, low, and close prices of each bin

print("VOLUME BARS SAMPLE:")
print(volume_bars.head())
print("\n")

print("VOLUME BAR PLOT:")
VolumeBarPlot = go.Figure(data=[go.Candlestick(x=volume_bars['date'],
                open=volume_bars['open'],
                high=volume_bars['high'],
                low=volume_bars['low'],
                close=volume_bars['close'])])
VolumeBarPlot.show()

KeyError: 'Volume'

In [5]:
"PREPROCESSING DATA"

# Creating a single column for Date and Time
tick_data['Date_Time'] = tick_data['Date'] + ' ' + tick_data['Time'] # Combining Date and Time columns
tick_data = tick_data.drop(['Date', 'Time'], axis=1) # Removing original Date and Time columns

# Calculating the transaction value
tick_data['Transaction'] = tick_data['Price'] * tick_data['Volume'] # Transaction value = Price * Volume

KeyError: 'Time'

In [ ]:
"FUNCTION DEFINITIONS"

def bin_data(data,bin_column,units):
    """
    Bins the data

    :param data: Pandas DataFrame of Tick Data
    :param bin_column: Column whose values are to be used to bin the data
    :param units: Number of units in a bar.
                  Tick Bars: Number of ticks per bar
                  Volume Bars: Number of shares traded per bar
                  Dollar Bars: Transaction size traded per bar

    :return: Pandas DataFrame with bins in a new column "Group"
    """
    num_bins = int(np.ceil(max(data.loc[:,bin_column])/units)) # number of bins depends on the range of bin_column and units
    bin_edges = [0+(b*units) for b in range(num_bins+1)] # bin_edges are equally spaced, starting at 0
    data['Group'] = pd.cut(x=data.loc[:,bin_column], bins=bin_edges, labels=list(range(num_bins))) # pandas cut function implemented
    return data

def extract_data(data):
    """
    Extracts open, high, low, and close values

    :param data: Pandas DataFrame of Tick Data

    :return: Pandas DataFrame with open, high, low, and close columns
    """
    date_time = data[['Date_Time', 'Group']].groupby('Group')['Date_Time'].last() # last date_time of each group (bin)
    ohlc = data[['Price', 'Group']].groupby('Group')['Price'].ohlc() # open, high, low, close prices of each group (bin)
    volume = data[['Volume', 'Group']].groupby('Group').sum() # sum of volumes of each group (bin)
    vwap = pd.DataFrame(data[['Transaction', 'Group']].groupby('Group').sum().values / volume.values) # volume-wtd avg price of each bin
    
    bars = pd.concat([date_time, ohlc, volume, vwap], axis=1) # combining the above calculated metrics
    bars.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'vwap'] # setting column names
    return bars